In [1]:
import torch
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os

from model.net import *
from utils.training import *
from data.data import *

## Experiment Config

In [5]:
model_path = os.getcwd()

args = {
    'USE_CUDA': True if torch.cuda.is_available() else False,
    'BATCH_SIZE': 256,
    'N_EPOCHS': 30,
    'LEARNING_RATE': 0.1,
    'MOMENTUM': 0.9,
    'DATASET_NAME':'mnist',
    'LAMBDA_recon': 1, #0.0005
    'LAMBDA_margin': 0,
}

## Model Loading

In [6]:
#Config for 49 16d vectors in the Primary Capsule. Set Softmax dimension to 0 in this case
class Config:
    def __init__(self):
        # CNN (cnn)
        self.cnn_in_channels = 1
        self.cnn_out_channels = 12
        self.cnn_kernel_size = 15

        # Primary Capsule (pc)
        self.pc_num_capsules = 16
        self.pc_in_channels = 12
        self.pc_out_channels = 1
        self.pc_kernel_size = 8
        self.pc_num_routes = 1 * 7 * 7

        # Digit Capsule 1 (dc)
        self.dc_num_capsules = 49
        self.dc_num_routes = 1 * 7 * 7
        self.dc_in_channels = 16
        self.dc_out_channels = 16
        
        # Digit Capsule 2 (dc)
        self.dc_2_num_capsules = 10
        self.dc_2_num_routes = 1 * 7 * 7
        self.dc_2_in_channels = 16
        self.dc_2_out_channels = 16

        # Decoder
        self.input_width = 28
        self.input_height = 28

torch.manual_seed(1)
config = Config()

net = CapsNet(args, config)
# capsule_net = torch.nn.DataParallel(capsule_net)
if args['USE_CUDA']:
    net = net.cuda()
    
#freeze All layers except Decoder
to_freeze = [net.conv_layer, net.primary_capsules, net.digit_capsules_1, net.digit_capsules_2]
for layer in to_freeze:
    for param in layer.parameters():
        param.requires_grad = False
    layer.eval()
    
net.load_state_dict(torch.load(os.path.join(model_path, 'CapsNetMNIST.pth'), map_location='cpu'))

<All keys matched successfully>

## Loading Dataset

In [7]:
trainloader, testloader = dataset(args)

## Training CapsuleNet

In [8]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()))

for e in range(1, args['N_EPOCHS'] + 1):
    train_CapsNet(net, optimizer, trainloader, e, args)
    test_CapsNet(net, testloader, e, args)

  1%|▏         | 3/235 [00:00<02:04,  1.86it/s]

Epoch: [1/30], Batch: [1/235], train accuracy: 0.992188, loss: 0.002686


 44%|████▍     | 103/235 [00:08<00:09, 13.20it/s]

Epoch: [1/30], Batch: [101/235], train accuracy: 0.976562, loss: 0.002621


 86%|████████▋ | 203/235 [00:16<00:02, 11.41it/s]

...

 86%|████████▋ | 203/235 [00:17<00:02, 11.44it/s]

Epoch: [30/30], Batch: [201/235], train accuracy: 0.968750, loss: 0.002529


100%|██████████| 235/235 [00:20<00:00, 11.42it/s]

Epoch: [30/30], train loss: 0.002506


Epoch: [30/30], test accuracy: 0.985100, loss: 0.649247


In [9]:
torch.save(net.state_dict(), "./CapsNetMNIST_Recon.pth")

In [9]:
for param in net.parameters():
    if(param.requires_grad==True):
        print(param.shape)

torch.Size([512, 160])
torch.Size([512])
torch.Size([1024, 512])
torch.Size([1024])
torch.Size([784, 1024])
torch.Size([784])


In [6]:
print(lambda p: p.requires_grad, net.parameters())

<function <lambda> at 0x7f7c076463b0> <generator object Module.parameters at 0x7f7c71d49ad0>


In [ ]:
#Config for 16 1d vectors in Capsule Layer. Set the Softmax Dimension to 1 in this case
# class Config:
#     def __init__(self, dataset='mnist'):
#         # CNN (cnn)
#         self.cnn_in_channels = 1
#         self.cnn_out_channels = 12
#         self.cnn_kernel_size = 15

#         # Primary Capsule (pc)
#         self.pc_num_capsules = 1
#         self.pc_in_channels = 12
#         self.pc_out_channels = 16
#         self.pc_kernel_size = 8
#         self.pc_num_routes = 16 * 7 * 7

#         # Digit Capsule 1 (dc)
#         self.dc_num_capsules = 49
#         self.dc_num_routes = 16 * 7 * 7
#         self.dc_in_channels = 1
#         self.dc_out_channels = 1 #16
        
#         # Digit Capsule 2 (dc)
#         self.dc_2_num_capsules = 10
#         self.dc_2_num_routes = 7 * 7
#         self.dc_2_in_channels = 1 #16
#         self.dc_2_out_channels = 16

#         # Decoder
#         self.input_width = 28
#         self.input_height = 28